# Humidity Controll

In [42]:
%reset -f

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import psychro as psy
import pandas as pd
import va_hum

![AHU](cool_CAV.svg)

In [44]:
# constants
l = 2496e3          # specific latent heat of vaporization [J/kg]
c = 1e3             # J/(kg·K), specific heat of dry air
ρ = 1.2             # kg/m3, density of air

# parameters
ϕ_0 = 0.8           # relative humidity of outside air [-]
θ_0 = 20            # temperature of outside Air [°C]
θ_6_sp = 22         # temperature setpoint of the room [°C]
ϕ_6_sp = 0.6        # relative humidity setpoint of the room [-]

α = 0.5             # mixing ratio of outdoor air [-]
m = 5.0             # mass flow rate of supply dry air [kg/s]
β = 0.1             # mixing ratio dehumidification [-]

UA = 936 # total transmittance [W/K]
m_i = 2.120 # infiltration mass flow [kg/s]
Q_aux_s = 4000 # auxiliary heat sensible [W]
Q_aux_l = 3000 # auxiliary  latent gain [W]

Kθ, Kw = 1e15, 1e15         # controller gain

θ_s0 = 5        # °C, initial guess for saturation temperature

In [45]:
# humidity ratios
w_6_sp = psy.w(θ_6_sp, ϕ_6_sp)  # humidity ratio setpoint of the room [kg/kg]
w_0 = psy.w(θ_0, ϕ_0)  # humidity ratio of outside air [kg/kg]

In [46]:
 # Define column names (unknowns)
columns_DH = [
    'θ1', 'w1', 'θ2', 'w2', 'θ3', 'w3', 'θ4', 'w4','θ5', 'w5','θ6', 'w6',
    'Qs_CC', 'Ql_CC', 'Qt_CC', 'Qs_HC', 'Qs_B', 'Ql_B'
]

# Define row names (equations) with 's' for sensible and 'l' for latent
rows_DH = [
    'MXs', 'MXl',                    # Mixing box equations
    'CCs', 'CCl', 'DH', 'DH_Q',      # Colling with dehumidification
    'MX2s', 'MX2l',                  # Mixing after dehumidification
    'HCs', 'HCl',                    # Heating coil equations
    'VHs', 'VHl',                    # Vapor humidifier equations

    'TZs', 'TZl',                   # Thermal zone equations
    'BLs', 'BLl',                   # Building equations
    'Kθ', 'Kw'                      # Controller equations
]

# Initialize coefficient matrix A and vector b
A_DH = pd.DataFrame(0.0, index=rows_DH, columns=columns_DH)
b_DH = pd.Series(0.0, index=rows_DH)


In [47]:
A_DH

,θ1,w1,θ2,w2,θ3,w3,θ4,w4,θ5,w5,θ6,w6,Qs_CC,Ql_CC,Qt_CC,Qs_HC,Qs_B,Ql_B
MXs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MXl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DH_Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MX2s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MX2l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# --------------------
# Mixing box equations
# --------------------

# sensible
A_DH.loc['MXs', 'θ1'] = m * c
A_DH.loc['MXs', 'θ6'] = -(1 - α) * m * c
b_DH.loc['MXs'] = α * m * c * θ_0

# latent
A_DH.loc['MXl', 'w1'] = m * l
A_DH.loc['MXl', 'w6'] = -(1 - α) * m * l
b_DH.loc['MXl'] = α * m * l * w_0

# ---------------------------------------
# Cooling with dehumidification equations
# ----------------------------------------

# sensible
A_DH.loc['CCs', 'θ2'] = α * m * c
A_DH.loc['CCs', 'θ1'] = -α * m * c
A_DH.loc['CCs', 'Qs_CC'] = 1
b_DH.loc['CCs'] = 0

# latent
A_DH.loc['CCl', 'w2'] = α * m * l
A_DH.loc['CCl', 'w1'] = -α * m * l
A_DH.loc['CCl', 'Ql_CC'] = 1
b_DH.loc['CCl'] = 0

# dehumidification acording saturation curve
A_DH.loc['DH', 'θ2'] = psy.wsp(θ_s0)
A_DH.loc['DH', 'w2'] = -1
b_DH.loc['DH'] = psy.wsp(θ_s0) * θ_s0 - psy.w(θ_s0, 1) 

# load equation
A_DH.loc['DH_Q', 'Qt_CC'] = -1
A_DH.loc['DH_Q', 'Qs_CC'] = 1
A_DH.loc['DH_Q', 'Ql_CC'] = 1
b_DH.loc['DH_Q'] = 0

# ----------------------------------------
# Mixing after dehumidification equations
# ----------------------------------------

# sensible
A_DH.loc['MX2s', 'θ3'] = m * c
A_DH.loc['MX2s', 'θ2'] = -β * m
A_DH.loc['MX2s', 'θ1'] = (1 - β) * m
b_DH.loc['MX2s'] = 0

# latent
A_DH.loc['MX2l', 'w3'] = m * l
A_DH.loc['MX2l', 'w2'] = -β * m 
A_DH.loc['MX2l', 'w1'] = -(1 - β) * m
b_DH.loc['MX2l'] = 0

# ----------------------
# Heating coil equations
# ----------------------

# sensible
A_DH.loc['HCs', 'θ4'] = m * c
A_DH.loc['HCs', 'θ3'] = -m * c
A_DH.loc['HCs', 'Qs_HC'] = 1
b_DH.loc['HCs'] = 0

# latent
A_DH.loc['HCl', 'w4'] = m * l
A_DH.loc['HCl', 'w3'] = -m * l
b_DH.loc['HCl'] = 0

# --------------------------
# Vapor humidifier equations
# --------------------------

# sensible
A_DH.loc['VHs', 'θ5'] = m * c
A_DH.loc['VHs', 'θ4'] = -m * c
b_DH.loc['VHs'] = 0

# latent
A_DH.loc['VHl', 'w5'] = m * l
A_DH.loc['VHl', 'w4'] = -m * l
b_DH.loc['VHl'] = 0

# ------------------------
# Thermal zone equations
# ------------------------

# sensible
A_DH.loc['TZs', 'θ6'] = m * c
A_DH.loc['TZs', 'θ5'] = -m * c
A_DH.loc['TZs', 'Qs_B'] = -1
b_DH.loc['TZs'] = 0

# latent
A_DH.loc['TZl', 'w6'] = m * l
A_DH.loc['TZl', 'w5'] = -m * l
A_DH.loc['TZl', 'Ql_B'] = -1
b_DH.loc['TZl'] = 0

# ------------------------  
# Building equations
# ------------------------

# sensible
A_DH.loc['BLs', 'Qs_B'] = 1
A_DH.loc['BLs', 'θ6'] = -(UA + m_i * c)
b_DH.loc['BLs'] = (UA * m_i * c) * θ_0 + Q_aux_s

# latent
A_DH.loc['BLl', 'Ql_B'] = 1
A_DH.loc['BLl', 'w6'] =-(m_i * l)
b_DH.loc['BLl'] = (m_i * l) * w_0 + Q_aux_l

# ------------------------
# Controller equations
# ------------------------

# temperature controller
A_DH.loc['Kθ', 'θ6'] = Kθ
A_DH.loc['Kθ', 'Qs_HC'] = 1
b_DH.loc['Kθ'] = Kθ * θ_6_sp

# humidity controller
A_DH.loc['Kw', 'w6'] = Kw
A_DH.loc['Kw', 'Ql_CC'] = 1
b_DH.loc['Kw'] = Kw * w_6_sp



In [49]:
A_DH

,θ1,w1,θ2,w2,θ3,w3,θ4,w4,θ5,w5,θ6,w6,Qs_CC,Ql_CC,Qt_CC,Qs_HC,Qs_B,Ql_B
MXs,5000.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.500000e+03,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
MXl,0.0,12480000.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,-6.240000e+06,0.0,0.0,0.0,0.0,0.0,0.0
CCs,-2500.0,0.0,2500.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.0,0.0,0.0,0.0,0.0,0.0
CCl,0.0,-6240000.0,0.00000,6240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0
DH,0.0,0.0,0.00038,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
DH_Q,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.0,1.0,-1.0,0.0,0.0,0.0
MX2s,4.5,0.0,-0.50000,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
MX2l,0.0,-4.5,0.00000,-0.5,0.0,12480000.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
HCs,0.0,0.0,0.00000,0.0,-5000.0,0.0,5000.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,1.0,0.0,0.0
HCl,0.0,0.0,0.00000,0.0,0.0,-12480000.0,0.0,12480000.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
b_DH

MXs     5.000000e+04
MXl     7.301279e+04
CCs     0.000000e+00
CCl     0.000000e+00
DH     -3.500718e-03
DH_Q    0.000000e+00
MX2s    0.000000e+00
MX2l    0.000000e+00
HCs     0.000000e+00
HCl     0.000000e+00
VHs     0.000000e+00
VHl     0.000000e+00
TZs     0.000000e+00
TZl     0.000000e+00
BLs     3.969040e+07
BLl     6.491484e+04
Kθ      2.200000e+16
Kw      9.895272e+12
dtype: float64

In [51]:
# Convert back to numpy arrays for solving
A_np = A_DH.to_numpy()
b_np = b_DH.to_numpy()

# Solution
x_DH = np.linalg.solve(A_np, b_np)

# Convert the solution array x_DH to a Pandas Series
solution_series = pd.Series(x_DH, index=columns_DH)
solution_series

θ1       2.100000e+01
w1       1.084064e-02
θ2       3.592294e+07
w2       1.365958e+04
θ3       3.592275e+03
w3       5.472626e-04
θ4      -7.929526e+03
w4       5.472626e-04
θ5      -7.929526e+03
w5       5.472626e-04
θ6       2.200000e+01
w6       9.980508e-03
Qs_CC   -8.980730e+10
Ql_CC   -8.523570e+10
Qt_CC   -1.750430e+11
Qs_HC    5.760901e+07
Qs_B     3.975763e+07
Ql_B     1.177269e+05
dtype: float64

# Humification

In [52]:
 # Define column names (unknowns)
columns_H = [
    'θ1', 'w1', 'θ2', 'w2', 'θ3', 'w3', 'θ4', 'w4','θ5', 'w5','θ6', 'w6',
    'Qs_HC', 'Ql_VC', 'Qs_B', 'Ql_B'
]

# Define row names (equations) with 's' for sensible and 'l' for latent
rows_H = [
    'MXs', 'MXl',                    # Mixing box equations
    'CCs', 'CCl',                    # Colling with dehumidification
    'MX2s', 'MX2l',                  # Mixing after dehumidification
    'HCs', 'HCl',                    # Heating coil equations
    'VHs', 'VHl',                    # Vapor humidifier equations

    'TZs', 'TZl',                   # Thermal zone equations
    'BLs', 'BLl',                   # Building equations
    'Kθ', 'Kw'                      # Controller equations
]

# Initialize coefficient matrix A and vector b
A_H = pd.DataFrame(0.0, index=rows_H, columns=columns_H)
b_H = pd.Series(0.0, index=rows_H)


In [53]:
# --------------------
# Mixing box equations
# --------------------

# sensible
A_H.loc['MXs', 'θ1'] = m * c
A_H.loc['MXs', 'θ6'] = -(1 - α) * m * c
b_H.loc['MXs'] = α * m * c * θ_0

# latent
A_H.loc['MXl', 'w1'] = m * l
A_H.loc['MXl', 'w6'] = -(1 - α) * m * l
b_H.loc['MXl'] = α * m * l * w_0

# ---------------------------------------
# Cooling with dehumidification equations
# ----------------------------------------

# sensible
A_H.loc['CCs', 'θ2'] = α * m * c
A_H.loc['CCs', 'θ1'] = -α * m * c
b_H.loc['CCs'] = 0

# latent
A_H.loc['CCl', 'w2'] = α * m * l
A_H.loc['CCl', 'w1'] = -α * m * l
b_H.loc['CCl'] = 0

# ----------------------------------------
# Mixing after dehumidification equations
# ----------------------------------------

# sensible
A_H.loc['MX2s', 'θ3'] = m * c
A_H.loc['MX2s', 'θ2'] = -β * m
A_H.loc['MX2s', 'θ1'] = -(1 - β) * m
b_H.loc['MX2s'] = 0

# latent
A_H.loc['MX2l', 'w3'] = m * l
A_H.loc['MX2l', 'w2'] = -β * m 
A_H.loc['MX2l', 'w1'] = -(1 - β) * m
b_H.loc['MX2l'] = 0

# ----------------------
# Heating coil equations
# ----------------------

# sensible
A_H.loc['HCs', 'θ4'] = m * c
A_H.loc['HCs', 'θ3'] = -m * c
A_H.loc['HCs', 'Qs_HC'] = 1
b_H.loc['HCs'] = 0
# latent
A_H.loc['HCl', 'w4'] = m * l
A_H.loc['HCl', 'w3'] = -m * l
b_H.loc['HCl'] = 0
# --------------------------
# Vapor humidifier equations
# --------------------------

# sensible
A_H.loc['VHs', 'θ5'] = m * c
A_H.loc['VHs', 'θ4'] = -m * c
b_H.loc['VHs'] = 0

# latent
A_H.loc['VHl', 'w5'] = m * l
A_H.loc['VHl', 'w4'] = -m * l
A_H.loc['VHl', 'Ql_VC'] = -1
b_H.loc['VHl'] = 0
# -----------------------
# Thermal zone equations
# ------------------------

# sensible
A_H.loc['TZs', 'θ6'] = m * c
A_H.loc['TZs', 'θ5'] = -m * c
A_H.loc['TZs', 'Qs_B'] = -1
b_H.loc['TZs'] = 0

# latent
A_H.loc['TZl', 'w6'] = m * l
A_H.loc['TZl', 'w5'] = -m * l
A_H.loc['TZl', 'Ql_B'] = -1
b_H.loc['TZl'] = 0
# ------------------------
# Building equations
# ------------------------

# sensible
A_H.loc['BLs', 'Qs_B'] = 1
A_H.loc['BLs', 'θ6'] = -(UA + m_i * c)
b_H.loc['BLs'] = (UA * m_i * c) * θ_0 + Q_aux_s

# latent
A_H.loc['BLl', 'Ql_B'] = 1
A_H.loc['BLl', 'w6'] = -(m_i * l)
b_H.loc['BLl'] = (m_i * l) * w_0 + Q_aux_l

# ------------------------
# Controller equations
# ------------------------

# temperature controller
A_H.loc['Kθ', 'θ6'] = Kθ
A_H.loc['Kθ', 'Qs_HC'] = 1
b_H.loc['Kθ'] = Kθ * θ_6_sp

# humidity controller
A_H.loc['Kw', 'w6'] = Kw
A_H.loc['Kw', 'Ql_VC'] = 1
b_H.loc['Kw'] = Kw * w_6_sp



In [54]:
A_H

,θ1,w1,θ2,w2,θ3,w3,θ4,w4,θ5,w5,θ6,w6,Qs_HC,Ql_VC,Qs_B,Ql_B
MXs,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.500000e+03,0.000000e+00,0.0,0.0,0.0,0.0
MXl,0.0,12480000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,-6.240000e+06,0.0,0.0,0.0,0.0
CCs,-2500.0,0.0,2500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0
CCl,0.0,-6240000.0,0.0,6240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0
MX2s,-4.5,0.0,-0.5,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0
MX2l,0.0,-4.5,0.0,-0.5,0.0,12480000.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0
HCs,0.0,0.0,0.0,0.0,-5000.0,0.0,5000.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.0,0.0,0.0,0.0
HCl,0.0,0.0,0.0,0.0,0.0,-12480000.0,0.0,12480000.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0
VHs,0.0,0.0,0.0,0.0,0.0,0.0,-5000.0,0.0,5000.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0
VHl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-12480000.0,0.0,12480000.0,0.000000e+00,0.000000e+00,0.0,-1.0,0.0,0.0


In [55]:
# Convert back to numpy arrays for solving
A_np = A_H.to_numpy()
b_np = b_H.to_numpy()

# Solution
x_H = np.linalg.solve(A_np, b_np)

# Convert the solution array x_DH to a Pandas Series
solution_series = pd.Series(x_H, index=columns_H)
solution_series

θ1       2.100000e+01
w1       1.079802e-02
θ2       2.100000e+01
w2       1.079802e-02
θ3       2.100000e-02
w3       4.326130e-09
θ4      -7.929526e+03
w4       4.326130e-09
θ5      -7.929526e+03
w5       4.981669e-04
θ6       2.200000e+01
w6       9.895272e-03
Qs_HC    3.964774e+07
Ql_VC    6.217068e+03
Qs_B     3.975763e+07
Ql_B     1.172759e+05
dtype: float64